# Statistics on the dataset

In [5]:
#import drill package and add it to the jupyter notebook environment
import sys
sys.path.append('../')
import dill

In [6]:
import os
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

source of the dataset

author = {Feng, Tao and Yue, Chuan},
title = {Visualizing and Interpreting RNN Models in URL-Based Phishing Detection},
year = {2020},
isbn = {9781450375689},
publisher = {Association for Computing Machinery},
address = {New York, NY, USA},
url = {https://doi.org/10.1145/3381991.3395602},
doi = {10.1145/3381991.3395602},
booktitle = {Proceedings of the 25th ACM Symposium on Access Control Models and Technologies},
pages = {13–24},
numpages = {12},
keywords = {visualization, interpretation, phishing detection, deep learning},
location = {Barcelona, Spain},
series = {SACMAT '20}
}```

Data import into a global dataframe

⚠ The data that we import is already encoded into numbers for their RNN analysis

In [7]:
import dill
dill_file="vonDataset20180426.dill"
with open(dill_file, 'rb') as f:
        pickleData=dill.load( f )
        train_x,train_y=pickleData["train_x"],pickleData["train_y"]
        val_x,val_y=pickleData["val_x"],pickleData["val_y"]
        test_x,test_y=pickleData["test_x"],pickleData["test_y"]
        char_to_int=pickleData["char_to_int"]
print("Feature Shapes:\n")
print("Train set: {}".format(train_x.shape), 
      " Validation set: {}".format(val_x.shape),
      " Test set: {}".format(test_x.shape))

Feature Shapes:

Train set: (1247489, 150)  Validation set: (155936, 150)  Test set: (155937, 150)


In [16]:
print(pickleData["char_to_int"])

{'|': 90, 'F': 51, 'd': 16, '!': 78, 'V': 67, '?': 48, '{': 94, 'P': 54, 'E': 49, '�': 88, 'B': 46, 'N': 60, 'a': 4, '5': 28, '8': 25, 'l': 15, ']': 97, 'G': 63, '1': 19, 's': 8, 'f': 26, '9': 33, 'M': 58, '0': 20, '补': 0, 'A': 43, 'L': 55, ' ': 89, ';': 57, 'J': 69, '<': 93, '\x82': 87, 'e': 3, 'r': 12, 'g': 24, '#': 84, '7': 34, 'H': 65, 'p': 10, 'Y': 68, 'x': 40, 'Ã': 79, '}': 96, 'X': 72, '-': 18, '`': 92, 'S': 52, 'C': 50, 'b': 21, '4': 31, '&': 44, '2': 22, 'i': 9, 'y': 36, 'T': 59, '_': 42, 'Q': 71, '6': 32, '$': 86, '*': 81, '\x83': 85, '\n': 30, 'O': 62, '~': 75, 'K': 73, '[': 98, 'D': 41, 'z': 47, 'I': 53, '3': 27, '.': 7, 'Z': 70, 'c': 6, 'o': 5, 'W': 66, '\x13': 95, ')': 83, 'h': 14, ':': 29, 'w': 17, 'u': 23, '%': 35, 'Â': 80, '未': 99, ',': 76, 'j': 45, 'v': 38, '=': 39, 'k': 37, '@': 77, '+': 74, 'U': 64, '/': 1, 'R': 61, 'q': 56, '(': 82, "'": 91, 'm': 13, 't': 2, 'n': 11}


In [12]:
print(train_x[10])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 14  2  2 10 29
  1  1 17 17 17  7  2  5 10 37 36  7  8 37  1 24 15  1 31 20 33 34 33 19
  1 19 25 28 33 33 27 28  1 64 13 12  3 15  4 18 13 23 18 21  4 21 37  4
 18 18 47  4  6  4 15 18 10  4  2 12  4  2 18  4 18 11  4 15  3 47 18 14
  5 18  8  5 37  5 38  4 15 18 18 62 16 14  4 15  9 15 18 11  3 37  4 15
  3 18 10 12  4 37  2  9 37 36 18  5 37 12  4 16  4 11  9  4 18 16  5  6
 14  5 16  6  5 38]


In [ ]:
abcde

Reconversion of the data into str

In [ ]:
#Transform the data back into string using the char_to_int dictionary
def int_to_char(x):
    res=''.join([list(char_to_int.keys())[list(char_to_int.values()).index(i)] for i in x])
    #remove from res the beginning of the string that had zero as a value
    # res=res[res.find('h'):]#is every string starting with 'h'?
    return res

#function that transforms the whole matrix into a list of strings
def matrix_to_list(x):
    res=[]
    for i in range(x.shape[0]):
        res.append(int_to_char(x[i]))
    return res

print(int_to_char(train_x[10]))

Data cleansing ?

Statistics ?